### bash script to calculate KE
To find the files:  

REF08 = /data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-REF08-S/5d/  
FUT08 = /data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-FUT08-S/5d/  
WORK = /data0/project/drakkar/USERS/jrieck/  

And CDFTOOLS are here

CDF = /home/ctalandi/TOOLS/CDFTOOLS/bin

In [ ]:
#%%bash 

cd $REF08
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};                
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-REF08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-REF08-KE/${fileK} $WORK/CREG12.L75-REF08-KE/${fileK};            
    done; 
done

cd $FUT08
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};               
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-FUT08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-FUT08-KE/${fileK} $WORK/CREG12.L75-FUT08-KE/${fileK};     
    done; 
done

CDFTOOLS to calculate Richardson number

In [ ]:
#%%bash

cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-REF08-RI/${fileR};              
    done; 
done

cd $WORK
for yyyy in {2051..2059}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-FUT08-RI/${fileR};              
    done; 
done

### calculate climatological seasonal cycle of U and V

In [ ]:
cd $WORK/CREG12.L75-REF08-UV_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $REF08/199[6-9]/CREG12.L75-REF08_y*.5d_gridU.nc $REF08/20??/CREG12.L75-REF08_y*.5d_gridU.nc U_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $REF08/199[6-9]/CREG12.L75-REF08_y*.5d_gridV.nc $REF08/20??/CREG12.L75-REF08_y*.5d_gridV.nc V_tmp${step}.nc
done
ncrcat -O -4 U_tmp??.nc CREG12.L75-REF08_1996-2015_climatology_gridU.nc
ncrcat -O -4 V_tmp??.nc CREG12.L75-REF08_1996-2015_climatology_gridV.nc

cd $WORK/CREG12.L75-FUT08-UV_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridU.nc U_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridV.nc V_tmp${step}.nc
done
ncrcat -O -4 U_tmp??.nc CREG12.L75-FUT08_2051-2070_climatology_gridU.nc
ncrcat -O -4 V_tmp??.nc CREG12.L75-FUT08_2051-2070_climatology_gridV.nc

test using xarray for extraction

In [ ]:
CONF = CONFS[1]
gridT_files = glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/206?/" + CONF 
                   + "_*.5d_gridT.nc") + glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/2070/" + CONF + "_*.5d_gridT.nc")
icemod_files = glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/206?/" + CONF 
                   + "_*.5d_icemod.nc") + glob("/data0/project/drakkar/CONFIGS/CREG12.L75/" + CONF + "-S/5d/2070/" + CONF + "_*.5d_icemod.nc")
KE_files = glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-KE/206?/" + CONF 
                + "_*.5d_KE.nc") + glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-KE/2070/" + CONF + "_*.5d_KE.nc")
RI_files = glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-RI/206?/" + CONF 
                + "_*.5d_RI.nc") + glob("/data0/project/drakkar/USERS/jrieck/" + CONF + "-RI/2070/" + CONF + "_*.5d_RI.nc")

for i in np.arange(0, len(moorn)):
    # seems like the extraction of profiles is a bit faster when selecting the ij-point already during the preprocessing
    def preproT(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.drop(["mldkz5", "botpres", "time_centered_bounds", "time_counter_bounds", "time_centered", "deptht_bounds"])
    def preproI(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.drop(["sitemp", "sisali", "siages", "snthic", "snvolu", "iicestru", "iicestrv", "sivelo", "sivolu", "sistre", 
                        "sidive", "sishea", "intstrx", "intstry", "time_centered_bounds", "time_counter_bounds", "time_centered"])
    def preproK(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.set_coords(("nav_lat", "nav_lon"))
    def preproR(ds):
        ds = ds.isel(x=moorx[i], y=moory[i])
        return ds.set_coords(("nav_lat", "nav_lon"))
    moorI = f'{moorx[i]:04}'
    moorJ = f'{moory[i]:04}'
    gridT = xr.decode_cf(xr.open_mfdataset(gridT_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproT, 
                                           decode_cf=False, coords="minimal", compat="override", parallel=True))
    icemod = xr.decode_cf(xr.open_mfdataset(icemod_files, chunks={"time_counter": 73}, preprocess=preproI, 
                                            decode_cf=False, coords="minimal", compat="override", parallel=True))
    KE = xr.decode_cf(xr.open_mfdataset(KE_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproK, 
                                        decode_cf=False, coords="minimal", compat="override", parallel=True))
    RI = xr.decode_cf(xr.open_mfdataset(RI_files, chunks={"time_counter": 73, "deptht": 75}, preprocess=preproR, 
                                        decode_cf=False, coords="minimal", compat="override", parallel=True))
    ds = xr.merge([gridT, icemod, KE, RI], compat="override").sel(time_counter=slice(str(years[CONF][0]) + "-01-01", str(years[CONF][-1]) + "-12-31"))
    years, datasets = zip(*ds.groupby("time_counter.year"))
    paths = [str("/data0/project/drakkar/USERS/jrieck/" + CONF + "-moorings/" + CONF + "_" + str(y) + "_moor" + moorn[i] + "_I" + moorI + "J" + moorJ 
             + "_gridT.nc") for y in years]
    xr.save_mfdataset(datasets, paths, engine="netcdf4", format="NETCDF4")
        

### bash scripts to extract points

This is slow as well. Using CDFTOOLS to extract from the 3D fields and ncks to extract from 2D fields. Not sure whether ncks could be faster for the 3D fields as well?

In [ ]:
#I=540
#J=1530
#number=001
#cd $REF08
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v votemper -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.votemper.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v vosaline -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.vosaline.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        $CDF/cdfprofile -f $file -v rhop_sig0 -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.rhop_sig0.profile; done
#done
#cd $WORK/CREG12.L75-REF08-moorings/
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.votemper.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_votemper.nc
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.vosaline.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_vosaline.nc
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.rhop_sig0.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_rhop_sig0.nc
#
#I=540
#J=1530
#number=002
#cd $REF08
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
#        ncks -O -d x,$I -d y,$J -v mldr10_1 $file $WORK/CREG12.L75-REF08-moorings/$file.mldr10_1.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
#        ncks -O -d x,$I -d y,$J -v siconc $file $WORK/CREG12.L75-REF08-moorings/$file.siconc.profile; done
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
#        ncks -O -d x,$I -d y,$J -v sithic $file $WORK/CREG12.L75-REF08-moorings/$file.sithic.profile; done
#done
#cd $WORK/CREG12.L75-REF08-moorings/
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*gridT.nc.mldr10_1.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_mldr10_1.nc
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.siconc.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_siconc.nc
#ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.sithic.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_sithic.nc
#
#I=540
#J=1530
#number=001
#cd $WORK/CREG12.L75-REF08-KE
#for year in {1996..2015}; do
#    echo $year
#    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_KE.nc; do 
#        $CDF/cdfprofile -f $file -v voke -IJ $I $J -o $WORK/CREG12.L75-REF08-moorings/$file.voke.profile; done
#done
#
#ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voke.profile CREG12.L75-REF08_y1996-2015_moor${number}_I0${I}J${J}_gridT_voke.nc

Using only ncks seems to work just fine. This way I can extract several variables at once. This takes a while but if I start extraction of each mooring point in parallel, it can be done overnight for one simulation.

In [ ]:
I=540
J=1530
number=001
for year in {1996..2015}; do
    echo $year
    cd $REF08
    echo "gridT"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_gridT.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v votemper,vosaline,rhop_sig0,mldr10_1 $file $WORK/CREG12.L75-REF08-moorings/$file.votemper.profile.I0${I}J${J}; done
    echo "icemod"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_icemod.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v siconc,sithic $file $WORK/CREG12.L75-REF08-moorings/$file.icemod.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-REF08-RI
    echo "voric"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_RI.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voric $file $WORK/CREG12.L75-REF08-moorings/$file.voric.profile.I0${I}J${J}; done
done

re-extract voke later because I recompute it right now

In [ ]:
I=540
J=1530
number=001
for year in {1996..2015}; do
    echo $year
    cd $WORK/CREG12.L75-REF08-KE
    echo "voke"
    for file in ${year}/CREG12.L75-REF08_y${year}m*.5d_KE.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voke $file $WORK/CREG12.L75-REF08-moorings/$file.voke.profile.I0${I}J${J}; done
done

In [ ]:
I=540
J=1530
number=001
cd $WORK/CREG12.L75-REF08-moorings/
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 *y????.5d_KE.nc.voke.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voke.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.gridT.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT.nc
ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.icemod.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_icemod.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voric.profile.I0${I}J${J} CREG12.L75-REF08_y1996-2015_moor${number}_I${I}J${J}_gridT_voric.nc

In [ ]:
I=540
J=1530
number=001
for year in {2051..2070}; do
    echo $year
    cd $FUT08
    echo "gridT"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_gridT.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v votemper,vosaline,rhop_sig0,mldr10_1 $file $WORK/CREG12.L75-FUT08-moorings/$file.votemper.profile.I0${I}J${J}; done
    echo "icemod"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_icemod.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v siconc,sithic $file $WORK/CREG12.L75-FUT08-moorings/$file.icemod.profile.I0${I}J${J}; done
    cd $WORK/CREG12.L75-FUT08-RI
    echo "voric"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_RI.nc; do 
        ncks -O -4 -d x,$I -d y,$J -v voric $file $WORK/CREG12.L75-FUT08-moorings/$file.voric.profile.I0${I}J${J}; done
done

re-extract voke later because I recompute it right no

In [ ]:
I=540
J=1530
number=001
for year in {2051..2070}; do
    echo $year
    cd $WORK/CREG12.L75-FUT08-KE
    echo "voke"
    for file in ${year}/CREG12.L75-FUT08_y${year}m*.5d_KE.nc; do
        ncks -O -4 -d x,$I -d y,$J -v voke $file $WORK/CREG12.L75-FUT08-moorings/$file.voke.profile.I0${I}J${J}; done
done

In [ ]:
I=540
J=1530
number=001
cd $WORK/CREG12.L75-FUT08-moorings/
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 *y????.5d_KE.nc.voke.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voke.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*gridT.nc.gridT.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT.nc
ncrcat -4 --cnk_dmn time_counter,1460 ????/*icemod.nc.icemod.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_icemod.nc
ncrcat -4 --cnk_dmn time_counter,1460 --cnk_dmn deptht,75 ????/*.nc.voric.profile.I0${I}J${J} CREG12.L75-FUT08_y2051-2070_moor${number}_I${I}J${J}_gridT_voric.nc

### End of preparations
---